In [3]:
# Voilà app: Segmentazione con Cellpose e salvataggio diametri
# Salva questo file come `voila_cell_diameter_app.py` o come notebook `.ipynb` e avvialo con:
#    pip install cellpose scikit-image matplotlib numpy pandas ipywidgets pillow
#    pip install voila
#    voila voila_cell_diameter_app.py
#
# L'app permette di caricare un'immagine, segmentarla con Cellpose, calcolare i diametri
# delle cellule e scaricare un CSV + l'immagine con i cerchi sovrapposti.

import io
import os
from cellpose import models
import numpy as np
import pandas as pd
from skimage.measure import regionprops
from skimage import io as skio
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, FileLink, clear_output

# ------- Helper functions -------

def read_image_from_bytes(b):
    """Prova a leggere un'immagine da bytes in vari formati e restituisce un numpy array.
    Ritorna immagine in formato (H, W) o (H, W, C) in uint8/float.
    """
    try:
        img = Image.open(io.BytesIO(b)).convert('RGB')
        arr = np.asarray(img)
        # Cellpose si aspetta array HxW o HxWxC
        return arr
    except Exception as e:
        raise RuntimeError(f"Impossibile leggere immagine: {e}")


def compute_cell_metrics(mask, pixel_size_um=None):
    props = regionprops(mask)
    rows = []
    for i, p in enumerate(props, start=1):
        area = p.area
        diameter_px = 2 * np.sqrt(area / np.pi)
        cy, cx = p.centroid
        minr, minc, maxr, maxc = p.bbox
        row = {
            'cell_id': i,
            'area_px': int(area),
            'diameter_px': float(diameter_px),
            'centroid_y': float(cy),
            'centroid_x': float(cx),
            'bbox_minr': int(minr),
            'bbox_minc': int(minc),
            'bbox_maxr': int(maxr),
            'bbox_maxc': int(maxc),
        }
        if pixel_size_um is not None:
            row['diameter_um'] = float(diameter_px * pixel_size_um)
            row['area_um2'] = float(area * (pixel_size_um ** 2))
        rows.append(row)
    df = pd.DataFrame(rows)
    return df


def draw_circles_on_image(image, df):
    # image: HxW or HxWx3
    fig, ax = plt.subplots(figsize=(8, 8))
    if image.ndim == 2:
        ax.imshow(image, cmap='gray')
    else:
        ax.imshow(image)

    for _, row in df.iterrows():
        circ = plt.Circle((row['centroid_x'], row['centroid_y']),
                          radius=row['diameter_px']/2,
                          edgecolor='lime', facecolor='none', lw=1.2)
        ax.add_patch(circ)
        ax.text(row['centroid_x'], row['centroid_y'], str(int(row['cell_id'])),
                color='white', fontsize=8, ha='center', va='center')

    ax.axis('off')
    plt.tight_layout()
    return fig


# ------- UI Widgets -------

uploader = widgets.FileUpload(accept='image/*', multiple=False)
run_button = widgets.Button(description='Segmenta e salva', button_style='success')
output = widgets.Output()
scale_input = widgets.FloatText(value=0.0, description='Pixel size (µm)',
                                tooltip='Inserisci la dimensione di un pixel in micrometri. Lascia 0 per disabilitare conversione.')
model_type_dd = widgets.Dropdown(options=['cyto','nuclei'], value='cyto', description='Model')
use_gpu_cb = widgets.Checkbox(value=False, description='Usa GPU (se disponibile)')

# Folder per output
OUT_DIR = 'voila_outputs'
os.makedirs(OUT_DIR, exist_ok=True)


# ------- Main callback -------

def on_run_click(b):
    with output:
        clear_output()
        if len(uploader.value) == 0:
            print("Per favore carica prima un'immagine.")
            return

        # Prendi il file caricato
        uploaded_file = uploader.value[0]  # il primo file caricato
        uploaded_filename = uploaded_file['name']
        content = uploaded_file['content']


        try:
            image = read_image_from_bytes(content)
        except Exception as e:
            print(e)
            return

        print(f"Immagine caricata: {uploaded_filename}, shape={image.shape}")

        # Carica il modello Cellpose (potrebbe richiedere tempo)
        print("Inizializzo il modello Cellpose...")
        try:
            model = models.Cellpose(gpu=bool(use_gpu_cb.value), model_type=model_type_dd.value)
        except Exception as e:
            print("Errore inizializzazione modello:", e)
            return

        print("Eseguo segmentazione (potrebbe impiegare qualche secondo)...")
        try:
            masks, flows, styles, diams = model.eval([image], diameter=None, channels=[0, 0])
        except Exception as e:
            print("Errore durante la segmentazione:", e)
            return

        mask = masks[0]
        print(f"Segmentazione completata. Cellpose diam estimate: {diams}")

        pixel_size = float(scale_input.value) if scale_input.value and scale_input.value > 0 else None

        df = compute_cell_metrics(mask, pixel_size_um=pixel_size)
        n_cells = len(df)
        print(f"Trovate {n_cells} cellule.")

        # Salva CSV
        csv_path = os.path.join(OUT_DIR, uploaded_filename + '_cell_diameters.csv')
        df.to_csv(csv_path, index=False)
        print('CSV salvato in:', csv_path)

        # Disegna immagine con cerchi
        fig = draw_circles_on_image(image, df)
        img_out_path = os.path.join(OUT_DIR, uploaded_filename + '_with_circles.png')
        fig.savefig(img_out_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print('Immagine con cerchi salvata in:', img_out_path)

        # Mostra risultato nella pagina
        display(FileLink(csv_path))
        display(FileLink(img_out_path))

        # Show small preview
        try:
            preview = skio.imread(img_out_path)
            fig2, ax2 = plt.subplots(figsize=(6,6))
            ax2.imshow(preview)
            ax2.axis('off')
            plt.show()
        except Exception:
            pass


run_button.on_click(on_run_click)

# ------- Layout -------

intro = widgets.HTML(
    "<h3>Voilà: Segmentazione Cellpose → CSV diametri + immagine</h3>"
    "<p>Carica un'immagine di cellule, seleziona il modello, premi <b>Segmenta e salva</b>. "
    "Otterrai un file CSV con le misure per cellula e un'immagine con cerchi sovrapposti.</p>"
)

controls = widgets.VBox([
    uploader,
    widgets.HBox([model_type_dd, use_gpu_cb, scale_input]),
    run_button
])

app = widgets.VBox([intro, controls, output])

display(app)
